This tutorial is for the calibration of an Xpad_flat on the Soleil Diffabs Diffractometer.

The calibration data is a scan on the delta motor with a CeO2 calibrant.

In [1]:
%matplotlib nbagg

# first the imports
from typing import Iterator, List, NamedTuple, Text, Tuple

import os
import functools

from collections import namedtuple
from math import pi

import numpy
import pylab
import pyFAI

from h5py import Dataset, File

from pyFAI.gui import jupyter

# local import
from soleil import *

In [2]:
# temporary until the ruch is ON
# ROOT = os.path.join("/home", "experiences", "instrumentation", "picca", "data", "99160066", "2016", "Run2")
ROOT = os.path.join("/nfs", "ruche-diffabs", "diffabs-users", "99160066", "2016", "Run2")
PUBLISHED = os.path.join("/home", "experiences", "instrumentation", "picca", "data", "99160066", "published-data")

In [3]:
# do a calibration with a well centered sample.

# TODO recuperer le point de départ à partir des ponis de référence.
# TODO ajouter le diagramme complet à la fin.
# TODO ajout du mask pour la calibration

json = os.path.join(PUBLISHED, "calibration", "calibration.json")

distance = Length(3.606331e-01)
poni1 = Length(1.314012e-01)
poni2 = Length(3.613907e-02)
rot1 = Angle(0)
rot2_scale = pi / 180.0  # convert degree to radian
rot2_offset = Angle(0.328063469162)  # radian
rot3 = Angle(7.19961198098e-07)

functions = (CalibrationFunctions("dist",
                                  "poni1",
                                  "poni2",
                                  "rot1",
                                  "rot2_scale * delta + rot2_offset",
                                  "rot3"),
             [ Parameter[Length]("dist", distance, (distance, distance)),
               Parameter[Length]("poni1", poni1, (poni1, poni1)),
               Parameter[Length]("poni2", poni2, (poni2, poni2)),
               Parameter[Angle]("rot1", rot1, (rot1, rot1)),
               Parameter[float]("rot2_scale", rot2_scale, (rot2_scale, rot2_scale)),
               Parameter[Angle]("rot2_offset", rot2_offset, (rot2_offset-0.5, rot2_offset+0.5)),
               Parameter[Angle]("rot3", rot3, (rot3, rot3))
            ]
           )

params = Calibration(os.path.join(PUBLISHED, "calibration"),
                     os.path.join(PUBLISHED, "calibration", "XRD18keV_26.nxs"),
                     DatasetPathWithAttribute("interpretation", b"image"),
                     DatasetPathContains("scan_data/actuator_1_1"),
                     [0, 3, 6],
                     Calibrant("CeO2"),
                     Detector("Xpad_flat"),
                     Wavelength(6.8877e-11),
                     functions             
                    )

In [4]:
# save all the ref as images in order to do the calibration with
# pyFAI-calib[2].
# temporary until pyFAI-calib2 can read from NeXuS files.

save_as_edf(params)

PermissionError: [Errno 13] Permission denied: '/home/experiences/instrumentation/picca/data/99160066/published-data/calibration/XRD18keV_26.nxs_00.edf'

In [ ]:
# execute the calibration

calibration(json, params, range(3))

In [ ]:
# plot the full diffractogram

def f(img: ndarray) -> ndarray:
    img = numpy.where(img > 2000, 0, img)
    return img

integrate(json, params, f)